# Задание

- Выберите набор данных (датасет) для решения задачи классификации или регресии.
- В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
- С использованием метода train_test_split разделите выборку на обучающую и тестовую.
- Обучите следующие ансамблевые модели:
  - две модели группы бэггинга (бэггинг или случайный лес или сверхслучайные деревья);
  - AdaBoost;
  - градиентный бустинг.
- Оцените качество моделей с помощью одной из подходящих для задачи метрик.Сравните качество полученных моделей.

Выбранный датасет (алмазы):
Содержание:
- Цена в долларах США ($326--$18,823)
- Карат  (0.2--5.01)
- Качество обрезки (Низкое, Хорошее, Очень хорошее, Премиум, Идеальное)
- Цвет бриллианта, от J (худший) до D (лучший)
- Измерение прозрачности бриллианта (I1 (худшее), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (лучшее))
- X длина в мм (0--10.74)
- Y ширина в мм (0--58.9)
- Z глубина в мм (0--31.8)
- Общий процент глубины = z / среднее(x, y) = 2 * z / (x + y) (43--79)
- Ширина вершины бриллианта относительно самой широкой точки (43--95)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="ticks")

In [3]:
data = pd.read_csv('diamonds.csv')

In [4]:
data = data.drop(columns=['id'])
data.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price        int64
x          float64
y          float64
z          float64
dtype: object

In [5]:
data.shape, data.price.shape

((53940, 10), (53940,))

In [6]:
data.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [7]:
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### Кодирование категориальных признаков

In [8]:
cut_order = {'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4}
color_order = {'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6}
clarity_order = {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}
data['cut_encoded'] = data['cut'].map(cut_order)
data['color_encoded'] = data['color'].map(color_order)
data['clarity_encoded'] = data['clarity'].map(clarity_order)
df_encoded = data.drop(columns=['cut', 'color', 'clarity'])
print(df_encoded)

       carat  depth  table  price     x     y     z  cut_encoded  \
0       0.23   61.5   55.0    326  3.95  3.98  2.43            4   
1       0.21   59.8   61.0    326  3.89  3.84  2.31            3   
2       0.23   56.9   65.0    327  4.05  4.07  2.31            1   
3       0.29   62.4   58.0    334  4.20  4.23  2.63            3   
4       0.31   63.3   58.0    335  4.34  4.35  2.75            1   
...      ...    ...    ...    ...   ...   ...   ...          ...   
53935   0.72   60.8   57.0   2757  5.75  5.76  3.50            4   
53936   0.72   63.1   55.0   2757  5.69  5.75  3.61            1   
53937   0.70   62.8   60.0   2757  5.66  5.68  3.56            2   
53938   0.86   61.0   58.0   2757  6.15  6.12  3.74            3   
53939   0.75   62.2   55.0   2757  5.83  5.87  3.64            4   

       color_encoded  clarity_encoded  
0                  5                1  
1                  5                2  
2                  5                4  
3                  1   

### Разделим выборку на обучающую и тестовую

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = df_encoded.drop("price", axis=1)
y = df_encoded["price"]

In [11]:
print(X.head(), "\n")
print(y.head())

   carat  depth  table     x     y     z  cut_encoded  color_encoded  \
0   0.23   61.5   55.0  3.95  3.98  2.43            4              5   
1   0.21   59.8   61.0  3.89  3.84  2.31            3              5   
2   0.23   56.9   65.0  4.05  4.07  2.31            1              5   
3   0.29   62.4   58.0  4.20  4.23  2.63            3              1   
4   0.31   63.3   58.0  4.34  4.35  2.75            1              0   

   clarity_encoded  
0                1  
1                2  
2                4  
3                3  
4                1   

0    326
1    326
2    327
3    334
4    335
Name: price, dtype: int64


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)

In [13]:
X_train.shape, y_train.shape,

((37758, 9), (37758,))

In [14]:
X_test.shape, y_test.shape

((16182, 9), (16182,))

### Обучение моделей

In [34]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [21]:
bagging_regressor = BaggingRegressor()
bagging_regressor.fit(X_train, y_train)

BaggingRegressor()

In [22]:
random_forest_regressor = RandomForestRegressor()
random_forest_regressor.fit(X_train, y_train)

RandomForestRegressor()

In [23]:
ada_boost_regressor = AdaBoostRegressor()
ada_boost_regressor.fit(X_train, y_train)

AdaBoostRegressor()

In [24]:
gradient_boosting_regressor = GradientBoostingRegressor()
gradient_boosting_regressor.fit(X_train, y_train)

GradientBoostingRegressor()

In [29]:
bagging_pred = bagging_regressor.predict(X_test)
print(bagging_pred)

[  950.8 12807.9  8356.2 ...  1127.8  7576.5  4235.9]


In [30]:
random_forest_pred = random_forest_regressor.predict(X_test)
print(random_forest_pred)

[  931.73 13205.07  8390.08 ...  1117.26  7452.68  4244.54]


In [31]:
ada_boost_pred = ada_boost_regressor.predict(X_test)
print(ada_boost_pred)

[ 1206.81102814 13051.09269718 10307.68137255 ...  1206.81102814
  8550.10004589  4186.34253308]


In [32]:
gradient_boosting_pred = gradient_boosting_regressor.predict(X_test)
print(gradient_boosting_pred)

[ 1028.67531217 13456.96439484  9085.2257314  ...  1116.44485684
  7803.85766126  3954.69303177]


In [33]:
bagging_mse = mean_squared_error(y_test, bagging_pred)
random_forest_mse = mean_squared_error(y_test, random_forest_pred)
ada_boost_mse = mean_squared_error(y_test, ada_boost_pred)
gradient_boosting_mse = mean_squared_error(y_test, gradient_boosting_pred)

In [35]:
bagging_r2_score = r2_score(y_test, bagging_pred)
random_forest_r2_score = r2_score(y_test, random_forest_pred)
ada_boost_r2_score = r2_score(y_test, ada_boost_pred)
gradient_boosting_r2_score = r2_score(y_test, gradient_boosting_pred)

In [36]:
print("Bagging MSE:", bagging_mse)
print("Random Forest MSE:", random_forest_mse)
print("AdaBoost MSE:", ada_boost_mse)
print("Gradient Boosting MSE:", gradient_boosting_mse)

Bagging MSE: 326341.08439436735
Random Forest MSE: 305018.64263461705
AdaBoost MSE: 1368676.7907716606
Gradient Boosting MSE: 408332.4673350993


In [37]:
print("Bagging r2_score:", bagging_r2_score)
print("Random Forest r2_score:", random_forest_r2_score)
print("AdaBoost r2_score:", ada_boost_r2_score)
print("Gradient Boosting r2_score:", gradient_boosting_r2_score)

Bagging r2_score: 0.9794313951287957
Random Forest r2_score: 0.9807753046161944
AdaBoost r2_score: 0.9137351272886313
Gradient Boosting r2_score: 0.9742636475199365


Видно, что максимальная точность будет у модели случайного леса - ~98%, потом у бэггинга - 97.9% и градиентного бустинга - 97.4%